#### Prerequisites


In [ ]:
from collections import deque
from math import sqrt
from typing import List, Optional


class TreeNode:
    def __init__(self, val: int = 0, left=None, right=None) -> None:
        self.val = val
        self.left = left
        self.right = right

    def __str__(self) -> str:
        s = ""
        lines, *_ = self._display_aux()
        for line in lines:
            s += line + "\n"
        return s

    def _display_aux(self):
        """Returns list of strings, width, height, and horizontal coordinate of the root."""
        # No child.
        if self.right is None and self.left is None:
            line = "%s" % self.val
            width = len(line)
            height = 1
            middle = width // 2
            return [line], width, height, middle

        # Only left child.
        if self.right is None:
            lines, n, p, x = self.left._display_aux()
            s = "%s" % self.val
            u = len(s)
            first_line = (x + 1) * " " + (n - x - 1) * "_" + s
            second_line = x * " " + "/" + (n - x - 1 + u) * " "
            shifted_lines = [line + u * " " for line in lines]
            return [first_line, second_line] + shifted_lines, n + u, p + 2, n + u // 2

        # Only right child.
        if self.left is None:
            lines, n, p, x = self.right._display_aux()
            s = "%s" % self.val
            u = len(s)
            first_line = s + x * "_" + (n - x) * " "
            second_line = (u + x) * " " + "\\" + (n - x - 1) * " "
            shifted_lines = [u * " " + line for line in lines]
            return [first_line, second_line] + shifted_lines, n + u, p + 2, u // 2

        # Two children.
        left, n, p, x = self.left._display_aux()
        right, m, q, y = self.right._display_aux()
        s = "%s" % self.val
        u = len(s)
        first_line = (x + 1) * " " + (n - x - 1) * "_" + s + y * "_" + (m - y) * " "
        second_line = (
            x * " " + "/" + (n - x - 1 + u + y) * " " + "\\" + (m - y - 1) * " "
        )
        if p < q:
            left += [n * " "] * (q - p)
        elif q < p:
            right += [m * " "] * (p - q)
        zipped_lines = zip(left, right)
        lines = [first_line, second_line] + [a + u * " " + b for a, b in zipped_lines]
        return lines, n + m + u, max(p, q) + 2, n + u // 2


def create_binary_tree_from_list(values: List[Optional[int]]) -> Optional[TreeNode]:
    if not values:
        return None

    root = TreeNode(values[0])
    queue = [root]
    i = 1

    while i < len(values):
        current = queue.pop(0)

        if values[i] is not None:
            current.left = TreeNode(values[i])
            queue.append(current.left)
        i += 1

        if i < len(values) and values[i] is not None:
            current.right = TreeNode(values[i])
            queue.append(current.right)
        i += 1

    return root

## 633. Sum of Square Numbers

    Difficuty - Medium
    Algos - Two Pointers, Binary Search

Given a non-negative integer `c`, decide whether there're two integers `a` and `b` such that <code>a<sup>2</sup> + b<sup>2</sup> = c</code>.

**Constraints:**

-   <code>0 <= c <= 2<sup>31</sup> - 1</code>


In [ ]:
class Solution:
    def judgeSquareSum(self, c: int) -> bool:
        if c < 3:
            return True

        low, high = 0, int(sqrt(c))

        while low <= high:
            current_sum = low**2 + high**2
            if current_sum == c:
                return True
            elif current_sum < c:
                low += 1
            else:
                high -= 1

        return False


if __name__ == "__main__":
    sol = Solution()
    cases = [{"c": 157}, {"c": 5}, {"c": 3}]
    for case in cases:
        print(sol.judgeSquareSum(case["c"]))

## 637. Average of Levels in Binary Tree

    Difficulty - Easy
    Topic - Binary Tree
    Algos - DFS, BFS

Given the `root` of a binary tree, return _the average value of the nodes on each level in the form of an array_. Answers within <code>10<sup>-5</sup></code> of the actual answer will be accepted.

**Constraints:**

-   The number of nodes in the tree is in the range <code>[1, 10<sup>4</sup>]</code>.
-   <code>-2<sup>31</sup> <= Node.val <= 2<sup>31</sup> - 1</code>


In [ ]:
class Solution:
    def averageOfLevels(self, root: Optional[TreeNode]) -> List[float]:
        if not root.left and not root.right:
            return [root.val]

        result = []
        queue = deque([root])

        while queue:
            level_size = len(queue)
            level_sum = 0
            count = 0

            for _ in range(level_size):
                node = queue.popleft()
                level_sum += node.val
                count += 1

                if node.left:
                    queue.append(node.left)
                if node.right:
                    queue.append(node.right)

            result.append(round(level_sum / count, 5))

        return result


if __name__ == "__main__":
    sol = Solution()
    cases = [{"root": [3, 9, 20, None, None, 15, 7]}, {"root": [3, 9, 20, 15, 7]}]
    for case in cases:
        root = create_binary_tree_from_list(case["root"])
        print(root)
        print("Average:\t", sol.averageOfLevels(root))
        print("-" * 35)

## 648. Replace Words

    Difficulty - Medium
    Topics - Array, String, Hash Table, Trie

In English, we have a concept called **root**, which can be followed by some other word to form another longer word - let's call this word **derivative**. For example, when the **root** `"help"` is followed by the word `"ful"`, we can form a derivative `"helpful"`.

Given a `dictionary` consisting of many **roots** and a `sentence` consisting of words separated by spaces, replace all the derivatives in the sentence with the **root** forming it. If a derivative can be replaced by more than one **root**, replace it with the **root** that has **the shortest length**.

Return _the_ `sentence` _after the replacement_.

**Constraints:**

-   `1 <= dictionary.length <= 1000`
-   `1 <= dictionary[i].length <= 100`
-   `dictionary[i]` consists of only lower-case letters.
-   <code>1 <= sentence.length <= 10<sup>6</sup></code>
-   `sentence` consists of only lower-case letters and spaces.
-   The number of words in `sentence` is in the range `[1, 1000]`
-   The length of each word in `sentence` is in the range `[1, 1000]`
-   Every two consecutive words in `sentence` will be separated by exactly one space.
-   `sentence` does not have leading or trailing spaces.


In [ ]:
class TrieNode:
    def __init__(self):
        self.children = {}
        self.is_end_of_word = False

    def __str__(self) -> str:
        s = ""
        lines, *_ = self._display_aux()
        for line in lines:
            s += line + "\n"
        return s

    def _display_aux(self):
        """Returns list of strings, width, height, and horizontal coordinate of the root."""
        if not self.children:
            return [""], 0, 1, 0

        lines = []
        height = 0
        width = 0
        middle = 0

        for char, child_node in self.children.items():
            child_lines, child_width, child_height, child_middle = (
                child_node._display_aux()
            )
            width += child_width
            height = max(height, child_height)
            lines.append((char, child_middle, child_lines))

        lines.sort(key=lambda x: x[1])
        middle = width // 2

        # Draw lines for each child node
        lines_display = []
        for char, _, child_lines in lines:
            line = " " * (middle - child_middle) + char + "".join(child_lines)
            lines_display.append(line)

        return lines_display, width, height + 1, middle


class Trie:
    def __init__(self):
        self.root = TrieNode()

    def insert(self, word):
        node = self.root
        for char in word:
            if char not in node.children:
                node.children[char] = TrieNode()
            node = node.children[char]
        node.is_end_of_word = True

    def search_shortest_prefix(self, word):
        node = self.root
        prefix = ""
        for char in word:
            if node.is_end_of_word:
                return prefix
            if char in node.children:
                prefix += char
                node = node.children[char]
            else:
                break
        return prefix if node.is_end_of_word else word

    def __str__(self):
        return str(self.root)


class Solution:
    def replaceWords(self, dictionary: List[str], sentence: str) -> str:
        trie = Trie()
        for word in dictionary:
            trie.insert(word)
        print(trie)

        result = []
        for word in sentence.split():
            result.append(trie.search_shortest_prefix(word))

        return " ".join(result)


if __name__ == "__main__":
    sol = Solution()
    cases = [
        {
            "dictionary": ["cat", "bat", "rat"],
            "sentence": "the cattle was rattled by the battery",
        },
        {"dictionary": ["a", "b", "c"], "sentence": "aadsfasf absbs bbab cadsfafs"},
    ]
    for case in cases:
        print(sol.replaceWords(case["dictionary"], case["sentence"]))